# **Inicio del AB FINAL**

## **Preparacion de los datos**

En este caso, se ha decidido crear este dataframe unificando todos los CSV en un único DataFrame y añadir una columna que identifique el origen (por ejemplo, source_id). Ya que la otra es crear un diccionario con cada csv separado, pero la custion es que las estructuras de columnas son muy parecidas y no paraece que se vaya a necesitar tratamientos específicos por fichero, si no mas bien en conjunto.

Por ello, se considera que es mas optima esta opción de unificar todos los datos en el mismo Dataframe

### Ruta de los *csv*

In [ ]:
# Instalar dependencias necesarias
import subprocess
import sys

packages = [
    "pandas",
    "numpy", 
    "openpyxl",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "scipy",
    "cmdstanpy",
    "pystan",
    "prophet"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✓ {package} ya está instalado")
    except ImportError:
        print(f"Instalando {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
            print(f"✓ {package} instalado correctamente")
        except subprocess.CalledProcessError:
            print(f"⚠️ Error instalando {package}, continuando...")

In [ ]:
import pandas as pd
from pathlib import Path
import re

### **Función de ingerta de datos**

En este caso, tras invesigar, encontre la funcion ***glob***, la cual busca un patron en comun al cual comparar en una ruta especificada y a partir de ahi, extrae todas las rutas cojn dicho patron. Por ello, para insertar todos los csv, hare una función que busque archivos acabados en "*.csv*".

Además, haré un bucle con un for para recorrer y comprobar que se hayan descargado y leido los datos correctamente.

#### ***Merge de los csv***

Una vez ya nos hemos asegurado de que todos los csv estan preparados para juntarlos en el mismo Dataframe, toca juntarlos propiamente en el mismo Dataframe para trabajar a posteriori con ellos.

Además, como vimos en el resultado de la celda anterior, hay muchas columnas llamadas Unnamed, las cuales no tienen nada de información dentro, por lo que a la hora de crear el Dataframe, lo solucionare eliminando todas estas columnas que no necesitamos y solo ensucian el Dataframe.



>  # Resultados de la celda anterior
    AguaEmbalsada_RioCofio_LaAcena.csv
    Columnas:['anio', 'mes', 'hec_cub', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']
    ---------------------------------------------------------------
    Archivo: AguaEmbalsada_RioGuadalix_Pedrezuela.csv
    Columnas: ['anio', 'mes', 'hec_cub', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13']quote



Además, para la nueva columna para el Dataframe, he decidido que el contenido de dicha columna, en vez de ser la ruta de donde se han importados los datos, sea mejor directamente el nombre del embalse de donde se han sacado los datos. Esto, lo he hecho mediante la siguiente forma:



>   # Añadir columna con el nombre del archivo:
    nombre = Path(archivo).stem
    embalse = nombre.split("_")[-1]          
    df["embalse"] = embalse









#### **Comparacion de los csv**

Una vez que ya se que se han leido todos los csv correctamente, quiero comparar las columnas de los distintos *csv*, ya que para poder fusionar en un mismo Dataframe todos los datos, necesitamos asegurarnos de que se van a apilar correectamente y no se van a generear columnbas indeseadas por que ciertas columnas se llamen de forma distinta, como por ejemplo:  

*   La columna se llama ***Año***
*   La columna se llama ***Fecha***

Además, he estado teniendo un mismo error a la hora de leer los datos, ya que me salia que:


    UnicodeDecodeError                        Traceback (most recent call last)
    /tmp/ipython-input-2309794888.py in <cell line: 0>()
          5
          6 for archivo in CSVs:
    ----> 7   df_temp = pd.read_csv(archivo, nrows=0, sep=";")
          8   columnas_por_archivo[archivo.split("/")[-1]] = list(df_temp.columns)
          9

    5 frames
    parsers.pyx in pandas._libs.parsers.TextReader.__cinit__()

    parsers.pyx in pandas._libs.parsers.TextReader._get_header()

    parsers.pyx in pandas._libs.parsers.TextReader._tokenize_rows()

    parsers.pyx in pandas._libs.parsers.TextReader._check_tokenize_status()

    parsers.pyx in pandas._libs.parsers.raise_parser_error()

    /usr/lib/python3.12/codecs.py in decode(self, input, final)

    UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 8059: invalid continuation byte *texto en cursiva*




Esto indica que alguno de los CSV no está en *UTF-8*, sino probablemente en *Latin-1 (ISO-8859-1)*, cosa que al parecer es muy comun en ficheros Españoles, por lo que he metido un segundo intento de lectura con ***encoding="latin-1"*** si falla con ***UTF-8.***


#### **Función de Ingesta**